In [1]:
!pip install pykalman

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 3.6 MB/s eta 0:00:00


In [193]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pykalman import KalmanFilter

In [194]:
data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')
data

<ipython-input-194-4b3032fe82e6>:1: DtypeWarning: Columns (15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')


,Date,AppName,Country,CONTINENT,AD_LAN,Store_Visit,Store_Acquisition,DAU,MAU,Bound,...,video_potentiality_games,day+_youtube,day+_games,Category_ID_youtube,Category_ID_games,similarity_score_games,Daily Average Rating,Total Average Rating,View_Count_previous_games,video_potentiality_previous_games
0,2021-07-29,퍼즐놀이,Argentina,중남미,스페인어,40,1,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,NaN,NaN,0.0,0.0
1,2021-07-29,퍼즐놀이,Australia,오세아니아,영어,12,1,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,NaN,NaN,0.0,0.0
2,2021-07-29,퍼즐놀이,Austria,유럽,영어,1,0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,NaN,NaN,0.0,0.0
3,2021-07-29,퍼즐놀이,Brazil,중남미,포르투갈어,102,10,0.0,0.0,2.0,...,0.000000,0.0,0.0,0.0,0.0,0,NaN,NaN,0.0,0.0
4,2021-07-29,퍼즐놀이,Canada,북미,영어,21,3,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291804,2024-08-09,여름휴가,Taiwan,아시아,중국어,23,8,0.0,0.0,24.0,...,12.728814,0.0,59.0,0.0,27.0,1,0.0,2.8750,0.0,0.0
291805,2024-08-09,여름휴가,United Arab Emirates,중동,아랍어,33,23,0.0,818.0,26.0,...,12.728814,0.0,59.0,0.0,27.0,1,0.0,3.8571,0.0,0.0
291806,2024-08-09,여름휴가,United Kingdom,유럽,영어,72,44,0.0,1920.0,51.0,...,12.728814,0.0,59.0,0.0,27.0,1,0.0,4.0000,0.0,0.0
291807,2024-08-09,여름휴가,United States,북미,영어,378,218,793.0,11026.0,314.0,...,12.728814,0.0,59.0,0.0,27.0,1,0.0,3.5416,0.0,0.0


In [195]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291809 entries, 0 to 291808
Data columns (total 35 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Date                               291809 non-null  object 
 1   AppName                            291809 non-null  object 
 2   Country                            291809 non-null  object 
 3   CONTINENT                          291809 non-null  object 
 4   AD_LAN                             291809 non-null  object 
 5   Store_Visit                        291809 non-null  int64  
 6   Store_Acquisition                  291809 non-null  int64  
 7   DAU                                291809 non-null  float64
 8   MAU                                291809 non-null  float64
 9   Bound                              291809 non-null  float64
 10  User acquisition                   291809 non-null  float64
 11  Impressions                        2918

In [196]:
# 국가와 앱 이름별로 그룹화하여 그룹별 평균값으로 결측치 채우기
grouped = data.groupby(['Country', 'AppName'])

# 그룹의 평균값으로 결측치 채우기
data['Daily Average Rating'] = grouped['Daily Average Rating'].transform(lambda x: x.fillna(x.mean()))
data['Total Average Rating'] = grouped['Total Average Rating'].transform(lambda x: x.fillna(x.mean()))

# 만약 그룹에 평균값도 없다면, 전체 평균으로 한 번 더 채워주는 코드 추가
data['Daily Average Rating'].fillna(data['Daily Average Rating'].mean(), inplace=True)
data['Total Average Rating'].fillna(data['Total Average Rating'].mean(), inplace=True)

# 결과를 확인하기 위해 데이터 출력
data.head()

<ipython-input-196-632a7fed75da>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Daily Average Rating'].fillna(data['Daily Average Rating'].mean(), inplace=True)
<ipython-input-196-632a7fed75da>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[co

,Date,AppName,Country,CONTINENT,AD_LAN,Store_Visit,Store_Acquisition,DAU,MAU,Bound,...,video_potentiality_games,day+_youtube,day+_games,Category_ID_youtube,Category_ID_games,similarity_score_games,Daily Average Rating,Total Average Rating,View_Count_previous_games,video_potentiality_previous_games
0,2021-07-29,퍼즐놀이,Argentina,중남미,스페인어,40,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.031373,2.860465,0.0,0.0
1,2021-07-29,퍼즐놀이,Australia,오세아니아,영어,12,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.193598,3.738161,0.0,0.0
2,2021-07-29,퍼즐놀이,Austria,유럽,영어,1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.193598,3.738161,0.0,0.0
3,2021-07-29,퍼즐놀이,Brazil,중남미,포르투갈어,102,10,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0,0.045126,2.904734,0.0,0.0
4,2021-07-29,퍼즐놀이,Canada,북미,영어,21,3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0.193598,3.738161,0.0,0.0


In [197]:
data.drop('Daily Average Rating',axis = 1, inplace = True)

In [198]:
data_1 = data.copy()

In [177]:
data.columns

Index(['Date', 'AppName', 'Country', 'CONTINENT', 'AD_LAN', 'Store_Visit',
       'Store_Acquisition', 'DAU', 'MAU', 'Bound', 'User acquisition',
       'Impressions', 'Est. earnings (KRW)', 'Observed eCPM (KRW)', 'Clicks',
       'CTR', 'Bids in auction (%)', 'Win rate', 'non_similar_youtube',
       'View_Count_games', 'similarity_score', 'View_Count_youtube',
       'Like_Count_youtube', 'Comment_Count_youtube',
       'video_potentiality_youtube', 'video_potentiality_games',
       'day+_youtube', 'day+_games', 'Category_ID_youtube',
       'Category_ID_games', 'similarity_score_games', 'Total Average Rating',
       'View_Count_previous_games', 'video_potentiality_previous_games'],
      dtype='object')

### user acquisition 자체 값 분리

### Category_ID_games 발견

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291809 entries, 0 to 291808
Data columns (total 34 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Date                               291809 non-null  object 
 1   AppName                            291809 non-null  object 
 2   Country                            291809 non-null  object 
 3   CONTINENT                          291809 non-null  object 
 4   AD_LAN                             291809 non-null  object 
 5   Store_Visit                        291809 non-null  int64  
 6   Store_Acquisition                  291809 non-null  int64  
 7   DAU                                291809 non-null  float64
 8   MAU                                291809 non-null  float64
 9   Bound                              291809 non-null  float64
 10  User acquisition                   291809 non-null  float64
 11  Impressions                        2918

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pykalman import KalmanFilter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# 추가 독립변수 생성: YouTube_Flag, Games_Flag
# YouTube 관련 열을 하나라도 포함하는 경우를 기준으로 YouTube_Flag 생성
data['YouTube_Flag'] = data.filter(like='Category_ID_youtube_').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

# Games 관련 열을 하나라도 포함하는 경우를 기준으로 Games_Flag 생성
data['Games_Flag'] = data.filter(like='Category_ID_games_').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['Est. earnings (KRW)', 'View_Count_games', 'similarity_score', 'View_Count_youtube',
                    'Like_Count_youtube', 'Comment_Count_youtube', 'video_potentiality_youtube',
                    'Total Average Rating']

# VIF 계산을 위한 데이터 준비
X = data[independent_vars + ['YouTube_Flag', 'Games_Flag'] + list(data.filter(like='AppName_').columns) +
         list(data.filter(like='Country_').columns) + list(data.filter(like='CONTINENT_').columns) +
         list(data.filter(like='AD_LAN_').columns)]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()
print("VIF가 낮은 독립변수들:", low_vif_features)

# Kalman Filter 설정
user_acquisition = data['User acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# VIF가 낮은 변수들을 독립변수로 사용하여 1단계 최소자승법 수행
X_1st_stage_perm = data[low_vif_features]
y_permanent = data['Permanent_Component']

# 1단계 최소자승법: Permanent_Component 예측
model_1st_stage_perm = sm.OLS(y_permanent, X_1st_stage_perm)
results_1st_stage_perm = model_1st_stage_perm.fit()

# 1단계 예측값 저장
data['Predicted_Permanent_Component'] = results_1st_stage_perm.predict(X_1st_stage_perm)

# 2단계 최소자승법: Permanent_Component를 예측한 결과를 사용해 회귀
X_2nd_stage_perm = data[['Predicted_Permanent_Component'] + low_vif_features]
model_2nd_stage_perm = sm.OLS(y_permanent, X_2nd_stage_perm)
results_2nd_stage_perm = model_2nd_stage_perm.fit()

print("Permanent Component - 1단계 OLS 결과")
print(results_1st_stage_perm.summary())
print("\nPermanent Component - 2단계 2SLS 결과")
print(results_2nd_stage_perm.summary())

# Temporary Component 분석

X_1st_stage_temp = data[low_vif_features]
y_temporary = data['Temporary_Component']

# 1단계 최소자승법: Temporary_Component 예측
model_1st_stage_temp = sm.OLS(y_temporary, X_1st_stage_temp)
results_1st_stage_temp = model_1st_stage_temp.fit()

# 1단계 예측값 저장
data['Predicted_Temporary_Component'] = results_1st_stage_temp.predict(X_1st_stage_temp)

# 2단계 최소자승법: Temporary_Component를 예측한 결과를 사용해 회귀
X_2nd_stage_temp = data[['Predicted_Temporary_Component'] + low_vif_features]
model_2nd_stage_temp = sm.OLS(y_temporary, X_2nd_stage_temp)
results_2nd_stage_temp = model_2nd_stage_temp.fit()

print("\nTemporary Component - 1단계 OLS 결과")
print(results_1st_stage_temp.summary())
print("\nTemporary Component - 2단계 2SLS 결과")

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


VIF 결과:
                               Feature        VIF
0                               const  17.176949
1                 Est. earnings (KRW)   1.679715
2                    View_Count_games  12.331822
3                    similarity_score   4.006387
4                  View_Count_youtube   1.606685
5                  Like_Count_youtube   1.073400
6               Comment_Count_youtube        NaN
7          video_potentiality_youtube   1.559009
8                Total Average Rating   1.016091
9                         Store_Visit  24.326332
10                  Store_Acquisition  84.505253
11                                DAU  17.400261
12                                MAU  22.155627
13                              Bound  43.920141
14                   User acquisition  70.347267
15                        Impressions  22.992028
16                Observed eCPM (KRW)   1.059998
17                             Clicks  19.007719
18                Bids in auction (%)   1.051855
19         

In [20]:
data.columns

Index(['const', 'Date', 'Store_Visit', 'Store_Acquisition', 'DAU', 'MAU',
       'Bound', 'User acquisition', 'Impressions', 'Est. earnings (KRW)',
       'Observed eCPM (KRW)', 'Clicks', 'CTR', 'Bids in auction (%)',
       'Win rate', 'non_similar_youtube', 'View_Count_games',
       'similarity_score', 'View_Count_youtube', 'Like_Count_youtube',
       'Comment_Count_youtube', 'video_potentiality_youtube',
       'video_potentiality_games', 'day+_youtube', 'day+_games',
       'Category_ID_youtube', 'Category_ID_games', 'similarity_score_games',
       'Total Average Rating', 'View_Count_previous_games',
       'video_potentiality_previous_games', 'YouTube_Flag', 'Games_Flag',
       'AppName_놀이공원', 'AppName_병원놀이', 'AppName_슈퍼마켓', 'AppName_여름휴가',
       'AppName_청소놀이', 'AppName_코코비 동물구조대', 'AppName_코코비 색칠놀이',
       'AppName_코코비 아기돌보기', 'AppName_코코비 월드1', 'AppName_코코비 월드2',
       'AppName_코코비 잠자기', 'AppName_퍼즐놀이', 'Country_Australia',
       'Country_Austria', 'Country_Brazil', 'Co

In [154]:
data = data_1.copy()

### 게임 영상의 언어 여부

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pykalman import KalmanFilter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User acquisition'])

# 추가 독립변수 생성: YouTube_Flag, Games_Flag
# YouTube 관련 열을 하나라도 포함하는 경우를 기준으로 YouTube_Flag 생성
data['YouTube_Flag'] = data.filter(like='Category_ID_youtube').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

# Games 관련 열을 하나라도 포함하는 경우를 기준으로 Games_Flag 생성
data['Games_Flag'] = data.filter(like='Category_ID_games').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

# 새로운 변수 생성: 한국어나 영어면 1, 그 이외는 0. 그리고 Category_ID_games가 0이면 그 값도 0으로 설정
def create_language_game_flag(row):
    # 언어가 한국어 또는 영어일 경우
    is_korean_or_english = (row['AD_LAN'] == '한국어' or row['AD_LAN'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

# apply를 통해 새로운 변수를 생성
data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)



# 독립변수 리스트 (수치형 데이터)
independent_vars = ['View_Count_games',
       'similarity_score', 'View_Count_youtube', 'Like_Count_youtube',
       'Comment_Count_youtube', 'video_potentiality_youtube',
       'Category_ID_youtube', 'Category_ID_games', 'similarity_score_games',
      'Total Average Rating']

# VIF 계산을 위한 데이터 준비
X = data[independent_vars + ['YouTube_Flag', 'Games_Flag', 'Language_Game_Flag'] +
         list(data.filter(like='AppName_').columns) + list(data.filter(like='Country_').columns) +
         list(data.filter(like='CONTINENT_').columns) + list(data.filter(like='AD_LAN_').columns)]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()
print("VIF가 낮은 독립변수들:", low_vif_features)

# Kalman Filter 설정
user_acquisition = data['User acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# VIF가 낮은 변수들을 독립변수로 사용하여 1단계 최소자승법 수행
X_1st_stage_perm = data[low_vif_features]
y_permanent = data['Permanent_Component']

# 1단계 최소자승법: Permanent_Component 예측
model_1st_stage_perm = sm.OLS(y_permanent, X_1st_stage_perm)
results_1st_stage_perm = model_1st_stage_perm.fit()

# 1단계 예측값 저장
data['Predicted_Permanent_Component'] = results_1st_stage_perm.predict(X_1st_stage_perm)

# 2단계 최소자승법: Permanent_Component를 예측한 결과를 사용해 회귀
X_2nd_stage_perm = data[['Predicted_Permanent_Component'] + low_vif_features]
model_2nd_stage_perm = sm.OLS(y_permanent, X_2nd_stage_perm)
results_2nd_stage_perm = model_2nd_stage_perm.fit()

print("Permanent Component - 1단계 OLS 결과")
print(results_1st_stage_perm.summary())
print("\nPermanent Component - 2단계 2SLS 결과")
print(results_2nd_stage_perm.summary())

# Temporary Component 분석

X_1st_stage_temp = data[low_vif_features]
y_temporary = data['Temporary_Component']

# 1단계 최소자승법: Temporary_Component 예측
model_1st_stage_temp = sm.OLS(y_temporary, X_1st_stage_temp)
results_1st_stage_temp = model_1st_stage_temp.fit()

# 1단계 예측값 저장
data['Predicted_Temporary_Component'] = results_1st_stage_temp.predict(X_1st_stage_temp)

# 2단계 최소자승법: Temporary_Component를 예측한 결과를 사용해 회귀
X_2nd_stage_temp = data[['Predicted_Temporary_Component'] + low_vif_features]
model_2nd_stage_temp = sm.OLS(y_temporary, X_2nd_stage_temp)
results_2nd_stage_temp = model_2nd_stage_temp.fit()

print("\nTemporary Component - 1단계 OLS 결과")
print(results_1st_stage_temp.summary())
print("\nTemporary Component - 2단계 2SLS 결과")

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


VIF 결과:
                        Feature        VIF
0                        const  19.008354
1             View_Count_games   1.394371
2             similarity_score  29.284701
3           View_Count_youtube   3.774523
4           Like_Count_youtube   1.092866
5        Comment_Count_youtube        NaN
6   video_potentiality_youtube   3.654655
7          Category_ID_youtube   5.283367
8            Category_ID_games  45.178013
9       similarity_score_games        inf
10        Total Average Rating   1.000466
11                YouTube_Flag  31.148887
12                  Games_Flag        inf
13          Language_Game_Flag   1.623191
VIF가 낮은 독립변수들: ['View_Count_games', 'View_Count_youtube', 'Like_Count_youtube', 'video_potentiality_youtube', 'Category_ID_youtube', 'Total Average Rating', 'Language_Game_Flag']
Permanent Component - 1단계 OLS 결과
                                 OLS Regression Results                                 
Dep. Variable:     Permanent_Component   R-squared (uncenter

## 24.102013

In [121]:
data = data_1.copy()

In [122]:
population = pd.read_excel('/content/drive/MyDrive/코코비_R&D팀/사용데이터/indicator_2024(수정).xlsx')
population

,KOR_NM,ENG_NM,YEAR,CODE,Population,Youth_Support_Rate,Youth_Ratio,TFR,Exchange Rate,GNI,...,CPI,Internet_Usage,Mobile_Usage_Rate,Mobile_Usage,GII,Leisure_cost,Leisure_cost_USD,Infant_Death,School_Life_Expectancy,Enrollment_PE
0,아르헨티나,Argentina,2021,AR,45277,0.37,0.23,1.89,48.15,11220,...,143.46,0.80,1.26,125.9,0.32,6296577.47,13443.47,9.3,2.36,1747663.00
1,오스트레일리아,Australia,2021,AU,25921,0.28,0.18,1.60,1.44,54900,...,119.80,0.90,1.10,110.3,0.09,103256.00,71705.56,2.9,1.69,539365.00
2,오스트리아,Austria,2021,AT,8922,0.22,0.14,1.47,0.89,51020,...,118.10,0.88,1.21,120.8,0.06,19326.00,21714.61,2.9,3.04,260088.00
3,브라질,Brazil,2021,BR,214326,0.29,0.20,1.64,3.94,9220,...,167.40,0.74,0.95,95.4,0.40,30993.00,7866.24,12.8,1.75,5217686.00
4,캐나다,Canada,2021,CA,38155,0.24,0.16,1.46,1.33,46600,...,116.80,0.95,0.92,91.6,0.07,99267.00,74636.84,4.3,2.34,953869.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,대만,Taiwan,2024,TW,23950,0.18,0.12,1.17,30.93,33664,...,103.00,0.86,1.27,126.6,0.12,6296122.25,30756.89,3.0,2.34,721639.86
116,아랍에미리트,United Arab Emirates,2024,AE,9592,0.18,0.15,1.44,3.67,48950,...,117.50,1.00,2.12,212.2,0.05,6295943.05,17002.21,4.8,1.87,314360.37
117,영국,United Kingdom,2024,GB,67961,0.27,0.17,1.58,0.78,48890,...,133.70,0.97,1.21,120.8,0.10,145025.00,179043.21,3.0,2.21,1636250.00
118,미국,United States,2024,US,341814,0.27,0.17,1.66,1.00,76370,...,134.20,0.92,1.10,110.2,0.18,1187254.00,1187254.00,4.8,2.12,8297381.00


In [123]:
population = population[population['YEAR']==2024][['ENG_NM','Population']]
population

,ENG_NM,Population
90,Argentina,46058
91,Australia,26699
92,Austria,8977
93,Brazil,217637
94,Canada,39107
95,Chile,19659
96,Colombia,52341
97,Costa Rica,5247
98,Ecuador,18377
99,Egypt,114484


In [142]:
data = pd.merge(data, population, left_on='Country', right_on='ENG_NM', how='left')
data.drop(columns=['ENG_NM'], inplace=True)
data

,Date,AppName,Country,CONTINENT,AD_LAN,Store_Visit,Store_Acquisition,DAU,MAU,Bound,...,video_potentiality_games,day+_youtube,day+_games,Category_ID_youtube,Category_ID_games,similarity_score_games,Total Average Rating,View_Count_previous_games,video_potentiality_previous_games,Population
0,2021-07-29,퍼즐놀이,Argentina,중남미,스페인어,40,1,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,2.860465,0.0,0.0,46058
1,2021-07-29,퍼즐놀이,Australia,오세아니아,영어,12,1,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,3.738161,0.0,0.0,26699
2,2021-07-29,퍼즐놀이,Austria,유럽,영어,1,0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,3.738161,0.0,0.0,8977
3,2021-07-29,퍼즐놀이,Brazil,중남미,포르투갈어,102,10,0.0,0.0,2.0,...,0.000000,0.0,0.0,0.0,0.0,0,2.904734,0.0,0.0,217637
4,2021-07-29,퍼즐놀이,Canada,북미,영어,21,3,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0,3.738161,0.0,0.0,39107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291804,2024-08-09,여름휴가,Taiwan,아시아,중국어,23,8,0.0,0.0,24.0,...,12.728814,0.0,59.0,0.0,27.0,1,2.875000,0.0,0.0,23950
291805,2024-08-09,여름휴가,United Arab Emirates,중동,아랍어,33,23,0.0,818.0,26.0,...,12.728814,0.0,59.0,0.0,27.0,1,3.857100,0.0,0.0,9592
291806,2024-08-09,여름휴가,United Kingdom,유럽,영어,72,44,0.0,1920.0,51.0,...,12.728814,0.0,59.0,0.0,27.0,1,4.000000,0.0,0.0,67961
291807,2024-08-09,여름휴가,United States,북미,영어,378,218,793.0,11026.0,314.0,...,12.728814,0.0,59.0,0.0,27.0,1,3.541600,0.0,0.0,341814


In [143]:
data['pop_user_acquisition'] = data['User acquisition'] / data['Population']
data

,Date,AppName,Country,CONTINENT,AD_LAN,Store_Visit,Store_Acquisition,DAU,MAU,Bound,...,day+_youtube,day+_games,Category_ID_youtube,Category_ID_games,similarity_score_games,Total Average Rating,View_Count_previous_games,video_potentiality_previous_games,Population,pop_user_acquisition
0,2021-07-29,퍼즐놀이,Argentina,중남미,스페인어,40,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,2.860465,0.0,0.0,46058,0.000022
1,2021-07-29,퍼즐놀이,Australia,오세아니아,영어,12,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,3.738161,0.0,0.0,26699,0.000000
2,2021-07-29,퍼즐놀이,Austria,유럽,영어,1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,3.738161,0.0,0.0,8977,0.000000
3,2021-07-29,퍼즐놀이,Brazil,중남미,포르투갈어,102,10,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0,2.904734,0.0,0.0,217637,0.000055
4,2021-07-29,퍼즐놀이,Canada,북미,영어,21,3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,3.738161,0.0,0.0,39107,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291804,2024-08-09,여름휴가,Taiwan,아시아,중국어,23,8,0.0,0.0,24.0,...,0.0,59.0,0.0,27.0,1,2.875000,0.0,0.0,23950,0.000418
291805,2024-08-09,여름휴가,United Arab Emirates,중동,아랍어,33,23,0.0,818.0,26.0,...,0.0,59.0,0.0,27.0,1,3.857100,0.0,0.0,9592,0.002919
291806,2024-08-09,여름휴가,United Kingdom,유럽,영어,72,44,0.0,1920.0,51.0,...,0.0,59.0,0.0,27.0,1,4.000000,0.0,0.0,67961,0.000912
291807,2024-08-09,여름휴가,United States,북미,영어,378,218,793.0,11026.0,314.0,...,0.0,59.0,0.0,27.0,1,3.541600,0.0,0.0,341814,0.000825


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from pykalman import KalmanFilter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import MinMaxScaler

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['pop_user_acquisition'])

# 추가 독립변수 생성: YouTube_Flag, Games_Flag
data['YouTube_Flag'] = data.filter(like='Category_ID_youtube').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
data['Games_Flag'] = data.filter(like='Category_ID_games').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

# 새로운 변수 생성: 한국어나 영어면 1, 그 이외는 0. 그리고 Category_ID_games가 0이면 그 값도 0으로 설정
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN'] == '한국어' or row['AD_LAN'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['View_Count_games', 'similarity_score', 'View_Count_youtube',
                    'Like_Count_youtube', 'Comment_Count_youtube', 'video_potentiality_youtube',
                    'Category_ID_youtube', 'Category_ID_games', 'similarity_score_games',
                    'Total Average Rating']

# VIF 계산을 위한 데이터 준비
X = data[independent_vars + ['YouTube_Flag', 'Games_Flag', 'Language_Game_Flag'] +
         list(data.filter(like='AppName_').columns) +
         list(data.filter(like='Country_').columns) +
         list(data.filter(like='CONTINENT_').columns) +
         list(data.filter(like='AD_LAN_').columns)]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# Min-Max Scaling 적용 (수치형 변수)
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# 상수항 추가 (회귀에 필요)
X_scaled = add_constant(X_scaled)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()
print("VIF가 낮은 독립변수들:", low_vif_features)

# Kalman Filter 설정
user_acquisition = data['pop_user_acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# VIF가 낮은 변수들을 독립변수로 사용하여 1단계 최소자승법 수행
X_1st_stage_perm = data[low_vif_features]
y_permanent = data['Permanent_Component']

# 1단계 최소자승법: Permanent_Component 예측
model_1st_stage_perm = sm.OLS(y_permanent, X_1st_stage_perm)
results_1st_stage_perm = model_1st_stage_perm.fit()

# 1단계 예측값 저장
data['Predicted_Permanent_Component'] = results_1st_stage_perm.predict(X_1st_stage_perm)

# 2단계 최소자승법: Permanent_Component를 예측한 결과를 사용해 회귀
X_2nd_stage_perm = data[['Predicted_Permanent_Component'] + low_vif_features]
model_2nd_stage_perm = sm.OLS(y_permanent, X_2nd_stage_perm)
results_2nd_stage_perm = model_2nd_stage_perm.fit()

print("Permanent Component - 1단계 OLS 결과")
print(results_1st_stage_perm.summary())
print("\nPermanent Component - 2단계 2SLS 결과")
print(results_2nd_stage_perm.summary())

# Temporary Component 분석
X_1st_stage_temp = data[low_vif_features]
y_temporary = data['Temporary_Component']

# 1단계 최소자승법: Temporary_Component 예측
model_1st_stage_temp = sm.OLS(y_temporary, X_1st_stage_temp)
results_1st_stage_temp = model_1st_stage_temp.fit()

# 1단계 예측값 저장
data['Predicted_Temporary_Component'] = results_1st_stage_temp.predict(X_1st_stage_temp)

# 2단계 최소자승법: Temporary_Component를 예측한 결과를 사용해 회귀
X_2nd_stage_temp = data[['Predicted_Temporary_Component'] + low_vif_features]
model_2nd_stage_temp = sm.OLS(y_temporary, X_2nd_stage_temp)
results_2nd_stage_temp = model_2nd_stage_temp.fit()

print("\nTemporary Component - 1단계 OLS 결과")
print(results_1st_stage_temp.summary())
print("\nTemporary Component - 2단계 2SLS 결과")

In [127]:
data['User acquisition'].mean()

145.0448478285454

In [128]:
data['CONTINENT'].unique()

array(['중남미', '오세아니아', '유럽', '북미', '중동', '아시아'], dtype=object)

In [129]:
# CONTINENT 열에서 '중동' 값을 가진 행을 삭제
data = data[data['CONTINENT'] != '중동']

In [130]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from pykalman import KalmanFilter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import MinMaxScaler

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['pop_user_acquisition'])

# 추가 독립변수 생성: YouTube_Flag, Games_Flag
data['YouTube_Flag'] = data.filter(like='Category_ID_youtube').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
data['Games_Flag'] = data.filter(like='Category_ID_games').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

# 새로운 변수 생성: 한국어나 영어면 1, 그 이외는 0. 그리고 Category_ID_games가 0이면 그 값도 0으로 설정
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN'] == '한국어' or row['AD_LAN'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['View_Count_games', 'similarity_score', 'View_Count_youtube',
                    'Like_Count_youtube', 'Comment_Count_youtube', 'video_potentiality_youtube',
                    'Category_ID_youtube', 'Category_ID_games', 'similarity_score_games',
                    'Total Average Rating']

# VIF 계산을 위한 데이터 준비
X = data[independent_vars + ['YouTube_Flag', 'Games_Flag', 'Language_Game_Flag'] +
         list(data.filter(like='AppName_').columns) +
         list(data.filter(like='Country_').columns) +
         list(data.filter(like='CONTINENT_').columns) +
         list(data.filter(like='AD_LAN_').columns)]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# Min-Max Scaling 적용 (수치형 변수)
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# 상수항 추가 (회귀에 필요)
X_scaled = add_constant(X_scaled)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()
print("VIF가 낮은 독립변수들:", low_vif_features)

# Kalman Filter 설정
user_acquisition = data['User acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# VIF가 낮은 변수들을 독립변수로 사용하여 1단계 최소자승법 수행
X_1st_stage_perm = data[low_vif_features]
y_permanent = data['Permanent_Component']

# 1단계 최소자승법: Permanent_Component 예측
model_1st_stage_perm = sm.OLS(y_permanent, X_1st_stage_perm)
results_1st_stage_perm = model_1st_stage_perm.fit()

# 1단계 예측값 저장
data['Predicted_Permanent_Component'] = results_1st_stage_perm.predict(X_1st_stage_perm)

# 2단계 최소자승법: Permanent_Component를 예측한 결과를 사용해 회귀
X_2nd_stage_perm = data[['Predicted_Permanent_Component'] + low_vif_features]
model_2nd_stage_perm = sm.OLS(y_permanent, X_2nd_stage_perm)
results_2nd_stage_perm = model_2nd_stage_perm.fit()

print("Permanent Component - 1단계 OLS 결과")
print(results_1st_stage_perm.summary())
print("\nPermanent Component - 2단계 2SLS 결과")
print(results_2nd_stage_perm.summary())

# Temporary Component 분석
X_1st_stage_temp = data[low_vif_features]
y_temporary = data['Temporary_Component']

# 1단계 최소자승법: Temporary_Component 예측
model_1st_stage_temp = sm.OLS(y_temporary, X_1st_stage_temp)
results_1st_stage_temp = model_1st_stage_temp.fit()

# 1단계 예측값 저장
data['Predicted_Temporary_Component'] = results_1st_stage_temp.predict(X_1st_stage_temp)

# 2단계 최소자승법: Temporary_Component를 예측한 결과를 사용해 회귀
X_2nd_stage_temp = data[['Predicted_Temporary_Component'] + low_vif_features]
model_2nd_stage_temp = sm.OLS(y_temporary, X_2nd_stage_temp)
results_2nd_stage_temp = model_2nd_stage_temp.fit()

print("\nTemporary Component - 1단계 OLS 결과")
print(results_1st_stage_temp.summary())
print("\nTemporary Component - 2단계 2SLS 결과")

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


VIF 결과:
                        Feature        VIF
0                        const  11.451471
1             View_Count_games   1.395587
2             similarity_score   2.167609
3           View_Count_youtube   1.681298
4           Like_Count_youtube   1.071759
5        Comment_Count_youtube        NaN
6   video_potentiality_youtube   1.688236
7          Category_ID_youtube   5.037559
8            Category_ID_games  66.279731
9       similarity_score_games   3.337890
10        Total Average Rating   1.001645
11                YouTube_Flag   6.002096
12                  Games_Flag  74.869227
13          Language_Game_Flag   1.797549
VIF가 낮은 독립변수들: ['View_Count_games', 'similarity_score', 'View_Count_youtube', 'Like_Count_youtube', 'video_potentiality_youtube', 'Category_ID_youtube', 'similarity_score_games', 'Total Average Rating', 'YouTube_Flag', 'Language_Game_Flag']
Permanent Component - 1단계 OLS 결과
                                 OLS Regression Results                                

In [132]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from pykalman import KalmanFilter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import MinMaxScaler

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data_ko_en = data_ko_en.dropna(subset=['pop_user_acquisition'])

# 추가 독립변수 생성: YouTube_Flag, Games_Flag
data_ko_en['YouTube_Flag'] = data_ko_en.filter(like='Category_ID_youtube').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
# data_ko_en['Games_Flag'] = data_ko_en.filter(like='Category_ID_games').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

# 새로운 변수 생성: 한국어나 영어면 1, 그 이외는 0. 그리고 Category_ID_games가 0이면 그 값도 0으로 설정
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN'] == '한국어' or row['AD_LAN'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

data_ko_en['Language_Game_Flag'] = data_ko_en.apply(create_language_game_flag, axis=1)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['View_Count_games', 'similarity_score', 'View_Count_youtube',
                    'Like_Count_youtube', 'Comment_Count_youtube', 'video_potentiality_youtube',
                    'Category_ID_youtube',  'similarity_score_games',
                    'Total Average Rating']

# VIF 계산을 위한 데이터 준비
X = data_ko_en[independent_vars + ['YouTube_Flag',  'Language_Game_Flag'] +
         list(data_ko_en.filter(like='AppName_').columns) +
         list(data_ko_en.filter(like='Country_').columns) +
         list(data_ko_en.filter(like='CONTINENT_').columns) +
         list(data_ko_en.filter(like='AD_LAN_').columns)]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# Min-Max Scaling 적용 (수치형 변수)
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# 상수항 추가 (회귀에 필요)
X_scaled = add_constant(X_scaled)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택하고, Language_Game_Flag는 무조건 포함
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()

# Language_Game_Flag를 강제로 포함
if 'Language_Game_Flag' not in low_vif_features:
    low_vif_features.append('Language_Game_Flag')

print("최종 선택된 독립변수들:", low_vif_features)

# Kalman Filter 설정
user_acquisition = data_ko_en['pop_user_acquisition'].values

# Kalman Filter를 적용할 데이터를 전체 데이터와 맞추기 위해 행 조정
# data_ko_en과 user_acquisition의 길이가 동일해야 함
data_ko_en_filtered = data_ko_en[~pd.isna(data_ko_en['pop_user_acquisition'])]  # user_acquisition의 결측값 제거 후 데이터 일치

# 칼만 필터 설정 및 실행
n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data_ko_en_filtered['Permanent_Component'] = state_means[:, 0]
data_ko_en_filtered['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data_ko_en_filtered = add_constant(data_ko_en_filtered)

# VIF가 낮은 변수들과 Language_Game_Flag를 독립변수로 사용하여 1단계 최소자승법 수행
X_1st_stage_perm = data_ko_en_filtered[low_vif_features]
y_permanent = data_ko_en_filtered['Permanent_Component']

# 1단계 최소자승법: Permanent_Component 예측
model_1st_stage_perm = sm.OLS(y_permanent, X_1st_stage_perm)
results_1st_stage_perm = model_1st_stage_perm.fit()

# 1단계 예측값 저장
data_ko_en_filtered['Predicted_Permanent_Component'] = results_1st_stage_perm.predict(X_1st_stage_perm)

# 2단계 최소자승법: Permanent_Component를 예측한 결과를 사용해 회귀
X_2nd_stage_perm = data_ko_en_filtered[['Predicted_Permanent_Component'] + low_vif_features]
model_2nd_stage_perm = sm.OLS(y_permanent, X_2nd_stage_perm)
results_2nd_stage_perm = model_2nd_stage_perm.fit()

print("Permanent Component - 1단계 OLS 결과")
print(results_1st_stage_perm.summary())
print("\nPermanent Component - 2단계 2SLS 결과")
print(results_2nd_stage_perm.summary())

# Temporary Component 분석
X_1st_stage_temp = data_ko_en_filtered[low_vif_features]
y_temporary = data_ko_en_filtered['Temporary_Component']

# 1단계 최소자승법: Temporary_Component 예측
model_1st_stage_temp = sm.OLS(y_temporary, X_1st_stage_temp)
results_1st_stage_temp = model_1st_stage_temp.fit()

# 1단계 예측값 저장
data_ko_en_filtered['Predicted_Temporary_Component'] = results_1st_stage_temp.predict(X_1st_stage_temp)

# 2단계 최소자승법: Temporary_Component를 예측한 결과를 사용해 회귀
X_2nd_stage_temp = data_ko_en_filtered[['Predicted_Temporary_Component'] + low_vif_features]
model_2nd_stage_temp = sm.OLS(y_temporary, X_2nd_stage_temp)
results_2nd_stage_temp = model_2nd_stage_temp.fit()

print("\nTemporary Component - 1단계 OLS 결과")
print(results_1st_stage_temp.summary())
print("\nTemporary Component - 2단계 2SLS 결과")

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


VIF 결과:
                        Feature        VIF
0                        const  14.185889
1             View_Count_games   1.354988
2             similarity_score   2.464854
3           View_Count_youtube   4.020458
4           Like_Count_youtube   1.079057
5        Comment_Count_youtube        NaN
6   video_potentiality_youtube   4.314310
7          Category_ID_youtube   5.053985
8       similarity_score_games   3.273405
9         Total Average Rating   1.002572
10                YouTube_Flag   6.356825
11          Language_Game_Flag   3.065838
최종 선택된 독립변수들: ['View_Count_games', 'similarity_score', 'View_Count_youtube', 'Like_Count_youtube', 'video_potentiality_youtube', 'Category_ID_youtube', 'similarity_score_games', 'Total Average Rating', 'YouTube_Flag', 'Language_Game_Flag']
Permanent Component - 1단계 OLS 결과
                                 OLS Regression Results                                 
Dep. Variable:     Permanent_Component   R-squared (uncentered):                   

## Log-log data_ko_en

In [119]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 252901 entries, 0 to 291808
Data columns (total 39 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Date                               252901 non-null  object 
 1   AppName                            252901 non-null  object 
 2   Country                            252901 non-null  object 
 3   CONTINENT                          252901 non-null  object 
 4   AD_LAN                             252901 non-null  object 
 5   Store_Visit                        252901 non-null  int64  
 6   Store_Acquisition                  252901 non-null  int64  
 7   DAU                                252901 non-null  float64
 8   MAU                                252901 non-null  float64
 9   Bound                              252901 non-null  float64
 10  User acquisition                   252901 non-null  float64
 11  Impressions                        252901 no

In [145]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from pykalman import KalmanFilter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import MinMaxScaler

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['pop_user_acquisition'])

# 추가 독립변수 생성: YouTube_Flag
data['YouTube_Flag'] = data.filter(like='Category_ID_youtube').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

# 새로운 변수 생성: 한국어나 영어면 1, 그 이외는 0. 그리고 Category_ID_games가 0이면 그 값도 0으로 설정
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN'] == '한국어' or row['AD_LAN'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['View_Count_games', 'similarity_score', 'View_Count_youtube',
                    'Like_Count_youtube', 'Comment_Count_youtube', 'video_potentiality_youtube',
                    'Category_ID_youtube', 'similarity_score_games', 'Total Average Rating']

# VIF 계산을 위한 데이터 준비
X = data[independent_vars + ['YouTube_Flag', 'Language_Game_Flag'] +
         list(data.filter(like='AppName_').columns) +
         list(data.filter(like='Country_').columns) +
         list(data.filter(like='CONTINENT_').columns) +
         list(data.filter(like='AD_LAN_').columns)]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# 로그 변환 (로그 변환 시 음수나 0값을 피하기 위해 모든 값에 소량의 상수 추가)
X_log = np.log(X + 1)

# 종속변수 log 변환
data['log_pop_user_acquisition'] = np.log(data['pop_user_acquisition'] + 1)

# Min-Max Scaling 적용 (수치형 변수)
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_log), columns=X.columns)

# 상수항 추가 (회귀에 필요)
X_scaled = add_constant(X_scaled)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택하고, Language_Game_Flag는 무조건 포함
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()

# Language_Game_Flag를 강제로 포함
if 'Language_Game_Flag' not in low_vif_features:
    low_vif_features.append('Language_Game_Flag')

print("최종 선택된 독립변수들:", low_vif_features)

# Kalman Filter 설정
user_acquisition = data['pop_user_acquisition'].values

# Kalman Filter를 적용할 데이터를 전체 데이터와 맞추기 위해 행 조정
data_filtered = data[~pd.isna(data['pop_user_acquisition'])]  # user_acquisition의 결측값 제거 후 데이터 일치

# 칼만 필터 설정 및 실행
n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data_filtered['Permanent_Component'] = state_means[:, 0]
data_filtered['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data_filtered = add_constant(data_filtered)

# 1단계 OLS 회귀 수행: Permanent_Component를 독립변수로 예측
X_1st_stage_perm = data_filtered[low_vif_features]
y_permanent = data_filtered['Permanent_Component']

model_1st_stage_perm = sm.OLS(y_permanent, X_1st_stage_perm)
results_1st_stage_perm = model_1st_stage_perm.fit()

# 1단계 예측값을 저장
data_filtered['Predicted_Permanent_Component'] = results_1st_stage_perm.predict(X_1st_stage_perm)

# **inf 또는 NaN 제거**: 'inf' 또는 'NaN' 값이 포함된 행을 제거합니다.
X_2nd_stage_perm = data_filtered[['Predicted_Permanent_Component'] + low_vif_features].replace([np.inf, -np.inf], np.nan).dropna()

# 종속변수와 독립변수의 인덱스를 일치시킵니다.
y_permanent = data_filtered['Permanent_Component'].reset_index(drop=True)
X_2nd_stage_perm = X_2nd_stage_perm.reset_index(drop=True)

# 2단계 최소자승법: Permanent_Component를 예측한 결과를 사용해 회귀
model_2nd_stage_perm = sm.OLS(y_permanent, X_2nd_stage_perm)
results_2nd_stage_perm = model_2nd_stage_perm.fit()

print("Permanent Component - 1단계 OLS 결과")
print(results_1st_stage_perm.summary())
print("\nPermanent Component - 2단계 2SLS 결과")
print(results_2nd_stage_perm.summary())

# Temporary Component 분석
X_1st_stage_temp = data_filtered[low_vif_features].replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop=True)
y_temporary = data_filtered['Temporary_Component'].reset_index(drop=True)

# 1단계 최소자승법: Temporary_Component 예측
model_1st_stage_temp = sm.OLS(y_temporary, X_1st_stage_temp)
results_1st_stage_temp = model_1st_stage_temp.fit()

# 1단계 예측값 저장
data_filtered['Predicted_Temporary_Component'] = results_1st_stage_temp.predict(X_1st_stage_temp)

# Temporary Component의 종속변수와 독립변수의 인덱스를 리셋하여 맞춥니다.
X_2nd_stage_temp = data_filtered[['Predicted_Temporary_Component'] + low_vif_features].replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop=True)
y_temporary = y_temporary.reset_index(drop=True)

# 2단계 최소자승법: Temporary_Component를 예측한 결과를 사용해 회귀
model_2nd_stage_temp = sm.OLS(y_temporary, X_2nd_stage_temp)
results_2nd_stage_temp = model_2nd_stage_temp.fit()

print("\nTemporary Component - 1단계 OLS 결과")
print(results_1st_stage_temp.summary())
print("\nTemporary Component - 2단계 2SLS 결과")

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


VIF 결과:
                        Feature        VIF
0                        const  15.184305
1             View_Count_games   4.504002
2             similarity_score  21.151180
3           View_Count_youtube  19.809014
4           Like_Count_youtube   1.760292
5        Comment_Count_youtube        NaN
6   video_potentiality_youtube  12.205755
7          Category_ID_youtube  26.765697
8       similarity_score_games   3.785587
9         Total Average Rating   1.001488
10                YouTube_Flag  43.986318
11          Language_Game_Flag   1.602950
최종 선택된 독립변수들: ['View_Count_games', 'Like_Count_youtube', 'similarity_score_games', 'Total Average Rating', 'Language_Game_Flag']
Permanent Component - 1단계 OLS 결과
                                 OLS Regression Results                                 
Dep. Variable:     Permanent_Component   R-squared (uncentered):                   0.430
Model:                             OLS   Adj. R-squared (uncentered):              0.430
Method:         

In [157]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from pykalman import KalmanFilter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import MinMaxScaler

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data_not_ko_en = data_not_ko_en.dropna(subset=['User acquisition'])

# 추가 독립변수 생성: YouTube_Flag
data_not_ko_en['YouTube_Flag'] = data_not_ko_en.filter(like='Category_ID_youtube').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

# 새로운 변수 생성: 한국어나 영어면 1, 그 이외는 0. 그리고 Category_ID_games가 0이면 그 값도 0으로 설정
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN'] == '한국어' or row['AD_LAN'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

data_not_ko_en['Language_Game_Flag'] = data_not_ko_en.apply(create_language_game_flag, axis=1)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['View_Count_games', 'similarity_score', 'View_Count_youtube',
                    'Like_Count_youtube', 'Comment_Count_youtube', 'video_potentiality_youtube',
                    'Category_ID_youtube', 'similarity_score_games', 'Total Average Rating']

# VIF 계산을 위한 데이터 준비
X = data_not_ko_en[independent_vars + ['YouTube_Flag', 'Language_Game_Flag'] +
         list(data_not_ko_en.filter(like='AppName_').columns) +
         list(data_not_ko_en.filter(like='Country_').columns) +
         list(data_not_ko_en.filter(like='CONTINENT_').columns) +
         list(data_not_ko_en.filter(like='AD_LAN_').columns)]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# 로그 변환 (로그 변환 시 음수나 0값을 피하기 위해 모든 값에 소량의 상수 추가)
X_log = np.log(X + 1)

# 종속변수 log 변환
data_not_ko_en['log_User acquisition'] = np.log(data_not_ko_en['User acquisition'] + 1)

# Min-Max Scaling 적용 (수치형 변수)
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_log), columns=X.columns)

# 상수항 추가 (회귀에 필요)
X_scaled = add_constant(X_scaled)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택하고, Language_Game_Flag는 무조건 포함
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()

# Language_Game_Flag를 강제로 포함
if 'Language_Game_Flag' not in low_vif_features:
    low_vif_features.append('Language_Game_Flag')

print("최종 선택된 독립변수들:", low_vif_features)

# Kalman Filter 설정
user_acquisition = data_not_ko_en['User acquisition'].values

# Kalman Filter를 적용할 데이터를 전체 데이터와 맞추기 위해 행 조정
data_filtered = data_not_ko_en[~pd.isna(data_not_ko_en['User acquisition'])]  # user_acquisition의 결측값 제거 후 데이터 일치

# 칼만 필터 설정 및 실행
n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data_filtered['Permanent_Component'] = state_means[:, 0]
data_filtered['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data_filtered = add_constant(data_filtered)

# 1단계 OLS 회귀 수행: Permanent_Component를 독립변수로 예측
X_1st_stage_perm = data_filtered[low_vif_features]
y_permanent = data_filtered['Permanent_Component']

model_1st_stage_perm = sm.OLS(y_permanent, X_1st_stage_perm)
results_1st_stage_perm = model_1st_stage_perm.fit()

# 1단계 예측값을 저장
data_filtered['Predicted_Permanent_Component'] = results_1st_stage_perm.predict(X_1st_stage_perm)

# **inf 또는 NaN 제거**: 'inf' 또는 'NaN' 값이 포함된 행을 제거합니다.
X_2nd_stage_perm = data_filtered[['Predicted_Permanent_Component'] + low_vif_features].replace([np.inf, -np.inf], np.nan).dropna()

# 종속변수와 독립변수의 인덱스를 일치시킵니다.
y_permanent = data_filtered['Permanent_Component'].reset_index(drop=True)
X_2nd_stage_perm = X_2nd_stage_perm.reset_index(drop=True)

# 2단계 최소자승법: Permanent_Component를 예측한 결과를 사용해 회귀
model_2nd_stage_perm = sm.OLS(y_permanent, X_2nd_stage_perm)
results_2nd_stage_perm = model_2nd_stage_perm.fit()

print("Permanent Component - 1단계 OLS 결과")
print(results_1st_stage_perm.summary())
print("\nPermanent Component - 2단계 2SLS 결과")
print(results_2nd_stage_perm.summary())

# Temporary Component 분석
X_1st_stage_temp = data_filtered[low_vif_features].replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop=True)
y_temporary = data_filtered['Temporary_Component'].reset_index(drop=True)

# 1단계 최소자승법: Temporary_Component 예측
model_1st_stage_temp = sm.OLS(y_temporary, X_1st_stage_temp)
results_1st_stage_temp = model_1st_stage_temp.fit()

# 1단계 예측값 저장
data_filtered['Predicted_Temporary_Component'] = results_1st_stage_temp.predict(X_1st_stage_temp)

# Temporary Component의 종속변수와 독립변수의 인덱스를 리셋하여 맞춥니다.
X_2nd_stage_temp = data_filtered[['Predicted_Temporary_Component'] + low_vif_features].replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop=True)
y_temporary = y_temporary.reset_index(drop=True)

# 2단계 최소자승법: Temporary_Component를 예측한 결과를 사용해 회귀
model_2nd_stage_temp = sm.OLS(y_temporary, X_2nd_stage_temp)
results_2nd_stage_temp = model_2nd_stage_temp.fit()

print("\nTemporary Component - 1단계 OLS 결과")
print(results_1st_stage_temp.summary())
print("\nTemporary Component - 2단계 2SLS 결과")

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


VIF 결과:
                        Feature        VIF
0                        const  12.382205
1             View_Count_games   3.798566
2             similarity_score  21.591284
3           View_Count_youtube  18.603558
4           Like_Count_youtube   1.701716
5        Comment_Count_youtube        NaN
6   video_potentiality_youtube  10.298742
7          Category_ID_youtube  26.544593
8       similarity_score_games   3.782373
9         Total Average Rating   1.002181
10                YouTube_Flag  38.144669
11          Language_Game_Flag        NaN
최종 선택된 독립변수들: ['View_Count_games', 'Like_Count_youtube', 'similarity_score_games', 'Total Average Rating', 'Language_Game_Flag']
Permanent Component - 1단계 OLS 결과
                                 OLS Regression Results                                 
Dep. Variable:     Permanent_Component   R-squared (uncentered):                   0.360
Model:                             OLS   Adj. R-squared (uncentered):              0.360
Method:         

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


ValueError: The indices for endog and exog are not aligned

### 강건성 확보

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pykalman import KalmanFilter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User acquisition'])

# 추가 독립변수 생성: YouTube_Flag, Games_Flag
data['YouTube_Flag'] = data.filter(like='Category_ID_youtube').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
data['Games_Flag'] = data.filter(like='Category_ID_games').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

# 새로운 변수 생성: 한국어나 영어면 1, 그 이외는 0. 그리고 Category_ID_games가 0이면 그 값도 0으로 설정
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN'] == '한국어' or row['AD_LAN'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['View_Count_games', 'similarity_score', 'View_Count_youtube', 'Like_Count_youtube',
                    'Comment_Count_youtube', 'video_potentiality_youtube', 'Category_ID_youtube',
                    'Category_ID_games', 'similarity_score_games', 'Total Average Rating']

# VIF 계산을 위한 데이터 준비
X = data[independent_vars + ['YouTube_Flag', 'Games_Flag', 'Language_Game_Flag'] +
         list(data.filter(like='AppName').columns) + list(data.filter(like='Country').columns) +
         list(data.filter(like='CONTINENT').columns) + list(data.filter(like='AD_LAN').columns)]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()
print("VIF가 낮은 독립변수들:", low_vif_features)

# Kalman Filter 설정
user_acquisition = data['User acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# VIF가 낮은 변수들을 독립변수로 사용하여 1단계 최소자승법 수행 (강건 표준 오차 사용)
X_1st_stage_perm = data[low_vif_features]
y_permanent = data['Permanent_Component']

# 1단계 최소자승법: Permanent_Component 예측 (강건한 표준 오차 적용)
model_1st_stage_perm = sm.OLS(y_permanent, X_1st_stage_perm)
results_1st_stage_perm = model_1st_stage_perm.fit(cov_type='HC3')

# 1단계 예측값 저장
data['Predicted_Permanent_Component'] = results_1st_stage_perm.predict(X_1st_stage_perm)

# 2단계 최소자승법: Permanent_Component를 예측한 결과를 사용해 회귀 (강건한 표준 오차 적용)
X_2nd_stage_perm = data[['Predicted_Permanent_Component'] + low_vif_features]
model_2nd_stage_perm = sm.OLS(y_permanent, X_2nd_stage_perm)
results_2nd_stage_perm = model_2nd_stage_perm.fit(cov_type='HC3')

print("Permanent Component - 1단계 OLS 결과")
print(results_1st_stage_perm.summary())
print("\nPermanent Component - 2단계 2SLS 결과")
print(results_2nd_stage_perm.summary())

# Temporary Component 분석 (강건한 표준 오차 적용)

X_1st_stage_temp = data[low_vif_features]
y_temporary = data['Temporary_Component']

# 1단계 최소자승법: Temporary_Component 예측 (강건한 표준 오차 적용)
model_1st_stage_temp = sm.OLS(y_temporary, X_1st_stage_temp)
results_1st_stage_temp = model_1st_stage_temp.fit(cov_type='HC3')

# 1단계 예측값 저장
data['Predicted_Temporary_Component'] = results_1st_stage_temp.predict(X_1st_stage_temp)

# 2단계 최소자승법: Temporary_Component를 예측한 결과를 사용해 회귀 (강건한 표준 오차 적용)
X_2nd_stage_temp = data[['Predicted_Temporary_Component'] + low_vif_features]
model_2nd_stage_temp = sm.OLS(y_temporary, X_2nd_stage_temp)
results_2nd_stage_temp = model_2nd_stage_temp.fit(cov_type='HC3')

print("\nTemporary Component - 1단계 OLS 결과")
print(results_1st_stage_temp.summary())
print("\nTemporary Component - 2단계 2SLS 결과")
print(results_2nd_stage_temp.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


VIF 결과:
                        Feature        VIF
0                        const  19.008354
1             View_Count_games   1.394371
2             similarity_score  29.284701
3           View_Count_youtube   3.774523
4           Like_Count_youtube   1.092866
5        Comment_Count_youtube        NaN
6   video_potentiality_youtube   3.654655
7          Category_ID_youtube   5.283367
8            Category_ID_games  45.178013
9       similarity_score_games        inf
10        Total Average Rating   1.000466
11                YouTube_Flag  31.148887
12                  Games_Flag        inf
13          Language_Game_Flag   1.623191
VIF가 낮은 독립변수들: ['View_Count_games', 'View_Count_youtube', 'Like_Count_youtube', 'video_potentiality_youtube', 'Category_ID_youtube', 'Total Average Rating', 'Language_Game_Flag']
Permanent Component - 1단계 OLS 결과
                                 OLS Regression Results                                 
Dep. Variable:     Permanent_Component   R-squared (uncenter

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 7
  warnings.warn('covariance of constraints does not have full '



Temporary Component - 1단계 OLS 결과
                                 OLS Regression Results                                 
Dep. Variable:     Temporary_Component   R-squared (uncentered):                   0.001
Model:                             OLS   Adj. R-squared (uncentered):              0.001
Method:                  Least Squares   F-statistic:                              21.51
Date:                 Sat, 19 Oct 2024   Prob (F-statistic):                    3.19e-29
Time:                         03:06:04   Log-Likelihood:                     -1.8703e+06
No. Observations:               292559   AIC:                                  3.741e+06
Df Residuals:                   292552   BIC:                                  3.741e+06
Df Model:                            7                                                  
Covariance Type:                   HC3                                                  
                                 coef    std err          z      P>|z|      

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 7
  warnings.warn('covariance of constraints does not have full '


In [ ]:
data.columns

Index(['Date', 'AppName', 'Country', 'CONTINENT', 'AD_LAN', 'Store_Visit',
       'Store_Acquisition', 'DAU', 'MAU', 'Bound', 'User acquisition',
       'Impressions', 'Est. earnings (KRW)', 'Observed eCPM (KRW)', 'Clicks',
       'CTR', 'Bids in auction (%)', 'Win rate', 'View_Count_games',
       'similarity_score', 'View_Count_youtube', 'Like_Count_youtube',
       'Comment_Count_youtube', 'video_potentiality_youtube',
       'Category_ID_youtube', 'Category_ID_games', 'similarity_score_games',
       'Total Average Rating', 'YouTube_Flag', 'Games_Flag'],
      dtype='object')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pykalman import KalmanFilter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User acquisition'])

# 추가 독립변수 생성: YouTube_Flag, Games_Flag
data['YouTube_Flag'] = data.filter(like='Category_ID_youtube').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
data['Games_Flag'] = data.filter(like='Category_ID_games').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

# 새로운 변수 생성: 한국어나 영어면 1, 그 이외는 0. 그리고 Category_ID_games가 0이면 그 값도 0으로 설정
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN'] == '한국어' or row['AD_LAN'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 범주형 변수를 더미 변수로 변환 (Country와 다른 범주형 변수들)
data = pd.get_dummies(data, columns=['Country', 'AppName', 'CONTINENT', 'AD_LAN'], drop_first=True)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['View_Count_games', 'similarity_score', 'View_Count_youtube', 'Like_Count_youtube',
                    'Comment_Count_youtube', 'video_potentiality_youtube', 'Category_ID_youtube',
                    'Category_ID_games', 'similarity_score_games', 'Total Average Rating',
                    'YouTube_Flag', 'Games_Flag', 'Language_Game_Flag']

# VIF 계산을 위한 데이터 준비
X = data[independent_vars +
         list(data.filter(like='Country').columns) +   # Country 더미 변수를 추가
         list(data.filter(like='AppName').columns) +
         list(data.filter(like='CONTINENT').columns) +
         list(data.filter(like='AD_LAN').columns)]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()
print("VIF가 낮은 독립변수들:", low_vif_features)

# Kalman Filter 설정
user_acquisition = data['User acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# VIF가 낮은 변수들을 독립변수로 사용하여 1단계 최소자승법 수행 (강건 표준 오차 사용)
X_1st_stage_perm = data[low_vif_features]
y_permanent = data['Permanent_Component']

# 1단계 최소자승법: Permanent_Component 예측 (강건한 표준 오차 적용)
model_1st_stage_perm = sm.OLS(y_permanent, X_1st_stage_perm)
results_1st_stage_perm = model_1st_stage_perm.fit(cov_type='HC3')

# 1단계 예측값 저장
data['Predicted_Permanent_Component'] = results_1st_stage_perm.predict(X_1st_stage_perm)

# 2단계 최소자승법: Permanent_Component를 예측한 결과를 사용해 회귀 (강건한 표준 오차 적용)
X_2nd_stage_perm = data[['Predicted_Permanent_Component'] + low_vif_features]
model_2nd_stage_perm = sm.OLS(y_permanent, X_2nd_stage_perm)
results_2nd_stage_perm = model_2nd_stage_perm.fit(cov_type='HC3')

print("Permanent Component - 1단계 OLS 결과")
print(results_1st_stage_perm.summary())
print("\nPermanent Component - 2단계 2SLS 결과")
print(results_2nd_stage_perm.summary())

# Temporary Component 분석 (강건한 표준 오차 적용)
X_1st_stage_temp = data[low_vif_features]
y_temporary = data['Temporary_Component']

# 1단계 최소자승법: Temporary_Component 예측 (강건한 표준 오차 적용)
model_1st_stage_temp = sm.OLS(y_temporary, X_1st_stage_temp)
results_1st_stage_temp = model_1st_stage_temp.fit(cov_type='HC3')

# 1단계 예측값 저장
data['Predicted_Temporary_Component'] = results_1st_stage_temp.predict(X_1st_stage_temp)

# 2단계 최소자승법: Temporary_Component를 예측한 결과를 사용해 회귀 (강건한 표준 오차 적용)
X_2nd_stage_temp = data[['Predicted_Temporary_Component'] + low_vif_features]
model_2nd_stage_temp = sm.OLS(y_temporary, X_2nd_stage_temp)
results_2nd_stage_temp = model_2nd_stage_temp.fit(cov_type='HC3')

print("\nTemporary Component - 1단계 OLS 결과")
print(results_1st_stage_temp.summary())
print("\nTemporary Component - 2단계 2SLS 결과")
print(results_2nd_stage_temp.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


VIF 결과:
                        Feature        VIF
0                        const  19.008354
1             View_Count_games   1.394371
2             similarity_score  29.284701
3           View_Count_youtube   3.774523
4           Like_Count_youtube   1.092866
5        Comment_Count_youtube        NaN
6   video_potentiality_youtube   3.654655
7          Category_ID_youtube   5.283367
8            Category_ID_games  45.178013
9       similarity_score_games        inf
10        Total Average Rating   1.000466
11                YouTube_Flag  31.148887
12                  Games_Flag        inf
13          Language_Game_Flag   1.623191
VIF가 낮은 독립변수들: ['View_Count_games', 'View_Count_youtube', 'Like_Count_youtube', 'video_potentiality_youtube', 'Category_ID_youtube', 'Total Average Rating', 'Language_Game_Flag']
Permanent Component - 1단계 OLS 결과
                                 OLS Regression Results                                 
Dep. Variable:     Permanent_Component   R-squared (uncenter

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 7
  warnings.warn('covariance of constraints does not have full '



Temporary Component - 1단계 OLS 결과
                                 OLS Regression Results                                 
Dep. Variable:     Temporary_Component   R-squared (uncentered):                   0.001
Model:                             OLS   Adj. R-squared (uncentered):              0.001
Method:                  Least Squares   F-statistic:                              21.51
Date:                 Sat, 19 Oct 2024   Prob (F-statistic):                    3.19e-29
Time:                         03:38:39   Log-Likelihood:                     -1.8703e+06
No. Observations:               292559   AIC:                                  3.741e+06
Df Residuals:                   292552   BIC:                                  3.741e+06
Df Model:                            7                                                  
Covariance Type:                   HC3                                                  
                                 coef    std err          z      P>|z|      

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 7
  warnings.warn('covariance of constraints does not have full '


In [ ]:
data['YouTube_Flag'].unique()

array([0])

In [166]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import statsmodels.api as sm
# from pykalman import KalmanFilter
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from statsmodels.tools.tools import add_constant
# from sklearn.preprocessing import StandardScaler

# # 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
# data = data.dropna(subset=['User acquisition'])

# # 새로운 YouTube_Flag 및 Games_Flag 생성: _youtube 또는 _games가 포함된 열 중 하나라도 0이 아니면 1을 넣음
# data['YouTube_Flag'] = data.filter(like='_youtube').apply(lambda row: 1 if (row != 0).any() else 0, axis=1)
# data['Games_Flag'] = data.filter(like='_games').apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# # 새로운 변수 생성: 한국어나 영어면 1, 그 이외는 0. 그리고 Category_ID_games가 0이면 그 값도 0으로 설정
# # def create_language_game_flag(row):
# #     is_korean_or_english = (row['AD_LAN'] == '한국어' or row['AD_LAN'] == '영어')
# #     if is_korean_or_english and row['Category_ID_games'] > 0:
# #         return 1
# #     else:
# #         return 0

# # data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# # 범주형 변수를 더미 변수로 변환 (Country와 다른 범주형 변수들)
# data = pd.get_dummies(data, columns=['Country', 'AppName', 'CONTINENT', 'AD_LAN'], drop_first=True)

# # 독립변수 리스트 (수치형 데이터)
# independent_vars = ['View_Count_games', 'similarity_score', 'View_Count_youtube', 'Like_Count_youtube',
#                     'Comment_Count_youtube', 'video_potentiality_youtube', 'Category_ID_youtube',
#                     'Category_ID_games', 'similarity_score_games', 'Total Average Rating',
#                     'YouTube_Flag', 'Games_Flag']

# # VIF 계산을 위한 데이터 준비
# X = data[independent_vars +
#          list(data.filter(like='Country').columns) +   # Country 더미 변수를 추가
#          list(data.filter(like='AppName').columns) +
#          list(data.filter(like='CONTINENT').columns) +
#          list(data.filter(like='AD_LAN').columns)]

# # 수치형 데이터만 선택하여 NaN값 처리
# X = X.select_dtypes(include=[np.number]).dropna()

# # 상수항 추가 (회귀에 필요)
# X = add_constant(X)

# # 스케일링 적용
# scaler = StandardScaler()
# X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# # VIF 계산
# vif_data = pd.DataFrame()
# vif_data["Feature"] = X_scaled.columns
# vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# # VIF 결과 출력
# print("VIF 결과:\n", vif_data)

# # VIF가 10 이하인 변수들만 선택
# low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()
# print("VIF가 낮은 독립변수들:", low_vif_features)

# # Kalman Filter 설정
# user_acquisition = data['User acquisition'].values

# n_timesteps = len(user_acquisition)
# n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# # 전이 행렬 (transition matrix)
# transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
#                               [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# # 관찰 행렬 (observation matrix)
# observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# # 초기 상태값 설정
# initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# # Kalman Filter 초기화
# kf = KalmanFilter(
#     transition_matrices=transition_matrix,
#     observation_matrices=observation_matrix,
#     initial_state_mean=initial_state_mean
# )

# # Kalman Filter를 사용해 상태 추정
# state_means, state_covariances = kf.filter(user_acquisition)

# # Permanent 및 Temporary Component 추출
# data['Permanent_Component'] = state_means[:, 0]
# data['Temporary_Component'] = state_means[:, 1]

# # 상수항을 추가 (Intercept)
# data = add_constant(data)

# # VIF가 낮은 변수들을 독립변수로 사용하여 1단계 최소자승법 수행 (강건 표준 오차 사용)
# X_1st_stage_perm = data[low_vif_features]
# y_permanent = data['Permanent_Component']

# # 1단계 최소자승법: Permanent_Component 예측 (강건한 표준 오차 적용)
# model_1st_stage_perm = sm.OLS(y_permanent, X_1st_stage_perm)
# results_1st_stage_perm = model_1st_stage_perm.fit(cov_type='HC3')

# # 1단계 예측값 저장
# data['Predicted_Permanent_Component'] = results_1st_stage_perm.predict(X_1st_stage_perm)

# # 2단계 최소자승법: Permanent_Component를 예측한 결과를 사용해 회귀 (강건한 표준 오차 적용)
# X_2nd_stage_perm = data[['Predicted_Permanent_Component'] + low_vif_features]
# model_2nd_stage_perm = sm.OLS(y_permanent, X_2nd_stage_perm)
# results_2nd_stage_perm = model_2nd_stage_perm.fit(cov_type='HC3')

# print("Permanent Component - 1단계 OLS 결과")
# print(results_1st_stage_perm.summary())
# print("\nPermanent Component - 2단계 2SLS 결과")
# print(results_2nd_stage_perm.summary())

# # Temporary Component 분석 (강건한 표준 오차 적용)
# X_1st_stage_temp = data[low_vif_features]
# y_temporary = data['Temporary_Component']

# # 1단계 최소자승법: Temporary_Component 예측 (강건한 표준 오차 적용)
# model_1st_stage_temp = sm.OLS(y_temporary, X_1st_stage_temp)
# results_1st_stage_temp = model_1st_stage_temp.fit(cov_type='HC3')

# # 1단계 예측값 저장
# data['Predicted_Temporary_Component'] = results_1st_stage_temp.predict(X_1st_stage_temp)

# # 2단계 최소자승법: Temporary_Component를 예측한 결과를 사용해 회귀 (강건한 표준 오차 적용)
# X_2nd_stage_temp = data[['Predicted_Temporary_Component'] + low_vif_features]
# model_2nd_stage_temp = sm.OLS(y_temporary, X_2nd_stage_temp)
# results_2nd_stage_temp = model_2nd_stage_temp.fit(cov_type='HC3')

# print("\nTemporary Component - 1단계 OLS 결과")
# print(results_1st_stage_temp.summary())
# print("\nTemporary Component - 2단계 2SLS 결과")

KeyError: "None of [Index(['Country', 'AppName', 'CONTINENT', 'AD_LAN'], dtype='object')] are in the [columns]"

In [1]:
data = pd.read_csv('/content/drive/MyDrive/코코비_R&D팀/사용데이터/final_data_24102012.csv')
data

NameError: name 'pd' is not defined

In [2]:
data.info()

NameError: name 'data' is not defined

In [ ]:

# 국가와 앱 이름별로 그룹화하여 그룹별 평균값으로 결측치 채우기
grouped = data.groupby(['Country', 'AppName'])

# 그룹의 평균값으로 결측치 채우기
data['Daily Average Rating'] = grouped['Daily Average Rating'].transform(lambda x: x.fillna(x.mean()))
data['Total Average Rating'] = grouped['Total Average Rating'].transform(lambda x: x.fillna(x.mean()))

# 만약 그룹에 평균값도 없다면, 전체 평균으로 한 번 더 채워주는 코드 추가
data['Daily Average Rating'].fillna(data['Daily Average Rating'].mean(), inplace=True)
data['Total Average Rating'].fillna(data['Total Average Rating'].mean(), inplace=True)

# 결과를 확인하기 위해 데이터 출력
data.head()

In [ ]:
data.drop(columns = 'Daily Average Rating',axis = 1, inplace = True)

In [ ]:
data.info()

In [ ]:
data.columns

In [3]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User_acquisition'])

# 새로운 YouTube_Flag 및 Games_Flag 생성: _youtube 또는 _games가 포함된 열 중 하나라도 0이 아니면 1을 넣음
data['YouTube_Flag'] = data.filter(like='_youtube').apply(lambda row: 1 if (row != 0).any() else 0, axis=1)
data['Games_Flag'] = data.filter(like='_games').apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# AppName과 Country를 더미 변환 전에 따로 저장
data['AppName_original'] = data['AppName']
data['Country_original'] = data['Country']
data['AD_LAN_original'] = data['AD_LAN']

# 변수 이름의 공백을 밑줄로 대체
data.columns = data.columns.str.replace(' ', '_')

# 범주형 변수를 더미 변수로 변환 (Country와 다른 범주형 변수들)
data = pd.get_dummies(data, columns=['Country', 'AppName', 'CONTINENT', 'AD_LAN'], drop_first=True)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['View_Count_games',
       'similarity_score', 'View_Count_youtube', 'Like_Count_youtube',
       'Comment_Count_youtube', 'video_potentiality_youtube',
       'video_potentiality_games', 'day+_youtube', 'day+_games',
       'Category_ID_youtube', 'Category_ID_games', 'similarity_score_games',
      'Total_Average_Rating',
       'View_Count_previous_games', 'video_potentiality_previous_games',
       'day+_unsimilar_youtube', 'video_potentiality_unsimilar_youtube',
       'Comment_Count_Count_unsimilar_youtube', 'View_Count_unsimilar_youtube',
       'Like_Count_unsimilar_youtube','YouTube_Flag','Games_Flag']

# VIF 계산을 위한 데이터 준비
X = data[independent_vars +
         list(data.filter(like='Country').columns) +   # Country 더미 변수를 추가
         list(data.filter(like='AppName').columns) +
         list(data.filter(like='CONTINENT').columns) +
         list(data.filter(like='AD_LAN').columns)]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# 스케일링 적용
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()
print("VIF가 낮은 독립변수들:", low_vif_features)

# Kalman Filter 설정
user_acquisition = data['User_acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# 패널 데이터를 위한 필요한 독립 변수만 선택 (low_vif_features 및 원본 AppName, Country, AD_LAN 사용)
panel_data = data[['Date', 'AppName_original', 'Country_original', 'Permanent_Component', 'Temporary_Component'] + low_vif_features + list(data.filter(like='AD_LAN').columns)]

# 결측값 제거
panel_data = panel_data.dropna()

# 변수명에서 +를 _로 교체
panel_data.columns = panel_data.columns.str.replace('+', '_')
low_vif_features = [feature.replace('+', '_') for feature in low_vif_features]

# 고정효과 패널 회귀 분석 (Permanent Component 분석)
model_perm = smf.ols('Permanent_Component ~ ' + ' + '.join(low_vif_features) + ' + C(AppName_original) + C(Country_original) + ' + ' + '.join(list(panel_data.filter(like="AD_LAN").columns)), data=panel_data).fit()

# 결과 출력
print("고정효과 패널 분석 결과 (Permanent Component):")
print(model_perm.summary())

# 고정효과 패널 회귀 분석 (Temporary Component 분석)
model_temp = smf.ols('Temporary_Component ~ ' + ' + '.join(low_vif_features) + ' + C(AppName_original) + C(Country_original) + ' + ' + '.join(list(panel_data.filter(like="AD_LAN").columns)), data=panel_data).fit()

# 결과 출력
print("\n고정효과 패널 분석 결과 (Temporary Component):")
print(model_temp.summary())

ModuleNotFoundError: No module named 'pykalman'

### 언어 요소 확인

In [384]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter
# 변수 이름의 공백을 밑줄로 대체
data.columns = data.columns.str.replace(' ', '_')

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User_acquisition'])

# 새로운 YouTube_Flag 및 Games_Flag 생성: _youtube 또는 _games가 포함된 열 중 하나라도 0이 아니면 1을 넣음
data['YouTube_Flag'] = data.filter(like='_youtube').apply(lambda row: 1 if (row != 0).any() else 0, axis=1)
data['Games_Flag'] = data.filter(like='_games').apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# AppName과 Country를 더미 변환 전에 따로 저장
data['AppName_original'] = data['AppName']
data['Country_original'] = data['Country']
data['AD_LAN_original'] = data['AD_LAN']



# 범주형 변수를 더미 변수로 변환 (Country와 다른 범주형 변수들)
data = pd.get_dummies(data, columns=['Country', 'AppName', 'CONTINENT', 'AD_LAN'], drop_first=True)

# Language_Game_Flag 생성 함수 정의
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN_original'] == '한국어' or row['AD_LAN_original'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

# Language_Game_Flag 적용
data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['View_Count_games',
       'similarity_score', 'View_Count_youtube', 'Like_Count_youtube',
       'Comment_Count_youtube', 'video_potentiality_youtube',
       'video_potentiality_games', 'day+_youtube', 'day+_games',
       'Category_ID_youtube', 'Category_ID_games', 'similarity_score_games',
       'Total_Average_Rating',
       'View_Count_previous_games', 'video_potentiality_previous_games',
       'day+_unsimilar_youtube', 'video_potentiality_unsimilar_youtube',
       'Comment_Count_Count_unsimilar_youtube', 'View_Count_unsimilar_youtube',
       'Like_Count_unsimilar_youtube','YouTube_Flag','Games_Flag', 'Language_Game_Flag']

# VIF 계산을 위한 데이터 준비
X = data[independent_vars +
         list(data.filter(like='Country').columns) +   # Country 더미 변수를 추가
         list(data.filter(like='AppName').columns) +
         list(data.filter(like='CONTINENT').columns) +
         list(data.filter(like='AD_LAN').columns)]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# 스케일링 적용
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()
print("VIF가 낮은 독립변수들:", low_vif_features)

# Kalman Filter 설정
user_acquisition = data['User_acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# 패널 데이터를 위한 필요한 독립 변수만 선택 (low_vif_features 및 원본 AppName, Country, AD_LAN 사용)
panel_data = data[['Date', 'AppName_original', 'Country_original', 'Permanent_Component', 'Temporary_Component'] + low_vif_features + list(data.filter(like='AD_LAN').columns)]

# 결측값 제거
panel_data = panel_data.dropna()

# 변수명에서 +를 _로 교체
panel_data.columns = panel_data.columns.str.replace('+', '_')
low_vif_features = [feature.replace('+', '_') for feature in low_vif_features]

# 고정효과 패널 회귀 분석 (Permanent Component 분석)
model_perm = smf.ols('Permanent_Component ~ ' + ' + '.join(low_vif_features) + ' + C(AppName_original) + C(Country_original) + ' + ' + '.join(list(panel_data.filter(like="AD_LAN").columns)), data=panel_data).fit()

# 결과 출력
print("고정효과 패널 분석 결과 (Permanent Component):")
print(model_perm.summary())

# 고정효과 패널 회귀 분석 (Temporary Component 분석)
model_temp = smf.ols('Temporary_Component ~ ' + ' + '.join(low_vif_features) + ' + C(AppName_original) + C(Country_original) + ' + ' + '.join(list(panel_data.filter(like="AD_LAN").columns)), data=panel_data).fit()

# 결과 출력
print("\n고정효과 패널 분석 결과 (Temporary Component):")
print(model_temp.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


KeyboardInterrupt: 

In [422]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pykalman import KalmanFilter

# 변수 이름의 공백을 밑줄로 대체
data.columns = data.columns.str.replace(' ', '_')

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['DAU'])

# 새로운 YouTube_Flag 및 Games_Flag 생성: _youtube 또는 _games가 포함된 열 중 하나라도 0이 아니면 1을 넣음
data['YouTube_Flag'] = data.filter(like='_youtube').apply(lambda row: 1 if (row != 0).any() else 0, axis=1)
data['Games_Flag'] = data.filter(like='_games').apply(lambda row: 1 if (row != 0).any() else 0, axis=1)

# AppName과 Country를 더미 변환 전에 따로 저장
data['AppName_original'] = data['AppName']
data['Country_original'] = data['Country']
data['AD_LAN_original'] = data['AD_LAN']

# 범주형 변수를 더미 변수로 변환 (Country와 다른 범주형 변수들)
data = pd.get_dummies(data, columns=['Country', 'AppName', 'CONTINENT', 'AD_LAN'], drop_first=True)

# Language_Game_Flag 생성 함수 정의
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN_original'] == '한국어' or row['AD_LAN_original'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

# Language_Game_Flag 적용
data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['View_Count_games',
       'similarity_score', 'View_Count_youtube', 'Like_Count_youtube',
       'Comment_Count_youtube', 'video_potentiality_youtube',
       'video_potentiality_games', 'day+_youtube', 'day+_games',
       'Category_ID_youtube', 'Category_ID_games', 'similarity_score_games',
       'Total_Average_Rating',
       'View_Count_previous_games', 'video_potentiality_previous_games',
       'day+_unsimilar_youtube', 'video_potentiality_unsimilar_youtube',
       'Comment_Count_Count_unsimilar_youtube', 'View_Count_unsimilar_youtube',
       'Like_Count_unsimilar_youtube','YouTube_Flag', 'Language_Game_Flag']

# 존재하지 않는 변수를 제거하는 필터링 처리
existing_vars = [var for var in independent_vars if var in data.columns]

# VIF 계산을 위한 데이터 준비
X = data[existing_vars +
         list(data.filter(like='Country').columns) +   # Country 더미 변수를 추가
         list(data.filter(like='AppName').columns) +
         list(data.filter(like='CONTINENT').columns) +
         list(data.filter(like='AD_LAN').columns)]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# 스케일링 적용
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()
print("VIF가 낮은 독립변수들:", low_vif_features)

# 변수명에서 +를 _로 교체 (Patsy에서 +가 오류를 유발할 수 있음)
low_vif_features = [feature.replace('+', '_') for feature in low_vif_features]

# 실제로 데이터프레임에 존재하는 변수만 선택
low_vif_features = [var for var in low_vif_features if var in data.columns]

# Kalman Filter 설정
user_acquisition = data['DAU'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# 패널 데이터를 위한 필요한 독립 변수만 선택 (low_vif_features 및 원본 AppName, Country, AD_LAN 사용)
panel_data = data[['Permanent_Component', 'Temporary_Component'] + low_vif_features + list(data.filter(like='AD_LAN').columns) + ['AppName_original', 'Country_original']]

# 결측값 제거
panel_data = panel_data.dropna()

# 고정효과 패널 회귀 분석 (Permanent Component 분석)
model_perm = smf.ols('Permanent_Component ~ ' + ' + '.join(low_vif_features) + ' + C(AppName_original) + C(Country_original)', data=panel_data).fit()

# 결과 출력 (Permanent Component 분석)
print("고정효과 패널 분석 결과 (Permanent Component):")
print(model_perm.summary())

# 고정효과 패널 회귀 분석 (Temporary Component 분석)
model_temp = smf.ols('Temporary_Component ~ ' + ' + '.join(low_vif_features) + ' + C(AppName_original) + C(Country_original)', data=panel_data).fit()

# 결과 출력 (Temporary Component 분석)
print("\n고정효과 패널 분석 결과 (Temporary Component):")
print(model_temp.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


VIF 결과:
                                   Feature        VIF
0                                   const        NaN
1                        View_Count_games  12.523887
2                        similarity_score   6.865352
3                      View_Count_youtube   1.817155
4                      Like_Count_youtube   1.080266
5                   Comment_Count_youtube        NaN
6              video_potentiality_youtube   1.773670
7                video_potentiality_games  13.497357
8                            day+_youtube   4.233000
9                              day+_games   8.948759
10                    Category_ID_youtube   2.742214
11                      Category_ID_games   3.406081
12                 similarity_score_games  11.331577
13                   Total_Average_Rating   1.001447
14              View_Count_previous_games   7.237008
15      video_potentiality_previous_games   7.238484
16                 day+_unsimilar_youtube   2.729754
17   video_potentiality_unsimilar_you

In [255]:
print(data.filter(like='AD_LAN').columns)

Index(['AD_LAN'], dtype='object')


## minmax scaling

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from pykalman import KalmanFilter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.preprocessing import StandardScaler

# 데이터의 결측값을 처리 (예: 특정 열에서 결측값을 제거)
data = data.dropna(subset=['User acquisition'])

# 추가 독립변수 생성: YouTube_Flag, Games_Flag
data['YouTube_Flag'] = data.filter(like='Category_ID_youtube').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)
data['Games_Flag'] = data.filter(like='Category_ID_games').sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

# 새로운 변수 생성: 한국어나 영어면 1, 그 이외는 0. 그리고 Category_ID_games가 0이면 그 값도 0으로 설정
def create_language_game_flag(row):
    is_korean_or_english = (row['AD_LAN'] == '한국어' or row['AD_LAN'] == '영어')
    if is_korean_or_english and row['Category_ID_games'] > 0:
        return 1
    else:
        return 0

data['Language_Game_Flag'] = data.apply(create_language_game_flag, axis=1)

# 범주형 변수를 더미 변수로 변환 (Country와 다른 범주형 변수들)
data = pd.get_dummies(data, columns=['Country', 'AppName', 'CONTINENT', 'AD_LAN'], drop_first=True)

# 독립변수 리스트 (수치형 데이터)
independent_vars = ['View_Count_games', 'similarity_score', 'View_Count_youtube', 'Like_Count_youtube',
                    'Comment_Count_youtube', 'video_potentiality_youtube', 'Category_ID_youtube',
                    'Category_ID_games', 'similarity_score_games', 'Total Average Rating',
                    'YouTube_Flag', 'Games_Flag', 'Language_Game_Flag']

# VIF 계산을 위한 데이터 준비
X = data[independent_vars +
         list(data.filter(like='Country').columns) +   # Country 더미 변수를 추가
         list(data.filter(like='AppName').columns) +
         list(data.filter(like='CONTINENT').columns) +
         list(data.filter(like='AD_LAN').columns)]

# 수치형 데이터만 선택하여 NaN값 처리
X = X.select_dtypes(include=[np.number]).dropna()

# 상수항 추가 (회귀에 필요)
X = add_constant(X)

# 스케일링 적용
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Feature"] = X_scaled.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled.values, i) for i in range(X_scaled.shape[1])]

# VIF 결과 출력
print("VIF 결과:\n", vif_data)

# VIF가 10 이하인 변수들만 선택
low_vif_features = vif_data[vif_data["VIF"] <= 10]["Feature"].tolist()
print("VIF가 낮은 독립변수들:", low_vif_features)

# Kalman Filter 설정
user_acquisition = data['User acquisition'].values

n_timesteps = len(user_acquisition)
n_dim_state = 2  # 두 개의 숨겨진 상태: permanent와 temporary

# 전이 행렬 (transition matrix)
transition_matrix = np.array([[1, 0.1],  # Permanent component: 약간의 변화를 허용
                              [0, 0.9]])  # Temporary component: 시간이 지남에 따라 감소

# 관찰 행렬 (observation matrix)
observation_matrix = np.array([[1, 1]])  # Permanent와 Temporary의 합을 관찰

# 초기 상태값 설정
initial_state_mean = [user_acquisition[0], 0]  # 첫 관측 값으로 시작

# Kalman Filter 초기화
kf = KalmanFilter(
    transition_matrices=transition_matrix,
    observation_matrices=observation_matrix,
    initial_state_mean=initial_state_mean
)

# Kalman Filter를 사용해 상태 추정
state_means, state_covariances = kf.filter(user_acquisition)

# Permanent 및 Temporary Component 추출
data['Permanent_Component'] = state_means[:, 0]
data['Temporary_Component'] = state_means[:, 1]

# 상수항을 추가 (Intercept)
data = add_constant(data)

# VIF가 낮은 변수들을 독립변수로 사용하여 1단계 최소자승법 수행 (강건 표준 오차 사용)
X_1st_stage_perm = data[low_vif_features]
y_permanent = data['Permanent_Component']

# 1단계 최소자승법: Permanent_Component 예측 (강건한 표준 오차 적용)
model_1st_stage_perm = sm.OLS(y_permanent, X_1st_stage_perm)
results_1st_stage_perm = model_1st_stage_perm.fit(cov_type='HC3')

# 1단계 예측값 저장
data['Predicted_Permanent_Component'] = results_1st_stage_perm.predict(X_1st_stage_perm)

# 2단계 최소자승법: Permanent_Component를 예측한 결과를 사용해 회귀 (강건한 표준 오차 적용)
X_2nd_stage_perm = data[['Predicted_Permanent_Component'] + low_vif_features]
model_2nd_stage_perm = sm.OLS(y_permanent, X_2nd_stage_perm)
results_2nd_stage_perm = model_2nd_stage_perm.fit(cov_type='HC3')

print("Permanent Component - 1단계 OLS 결과")
print(results_1st_stage_perm.summary())
print("\nPermanent Component - 2단계 2SLS 결과")
print(results_2nd_stage_perm.summary())

# Temporary Component 분석 (강건한 표준 오차 적용)
X_1st_stage_temp = data[low_vif_features]
y_temporary = data['Temporary_Component']

# 1단계 최소자승법: Temporary_Component 예측 (강건한 표준 오차 적용)
model_1st_stage_temp = sm.OLS(y_temporary, X_1st_stage_temp)
results_1st_stage_temp = model_1st_stage_temp.fit(cov_type='HC3')

# 1단계 예측값 저장
data['Predicted_Temporary_Component'] = results_1st_stage_temp.predict(X_1st_stage_temp)

# 2단계 최소자승법: Temporary_Component를 예측한 결과를 사용해 회귀 (강건한 표준 오차 적용)
X_2nd_stage_temp = data[['Predicted_Temporary_Component'] + low_vif_features]
model_2nd_stage_temp = sm.OLS(y_temporary, X_2nd_stage_temp)
results_2nd_stage_temp = model_2nd_stage_temp.fit(cov_type='HC3')

print("\nTemporary Component - 1단계 OLS 결과")
print(results_1st_stage_temp.summary())
print("\nTemporary Component - 2단계 2SLS 결과")
print(results_2nd_stage_temp.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss
/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


VIF 결과:
                        Feature        VIF
0                        const        NaN
1             View_Count_games   1.394371
2             similarity_score  29.284701
3           View_Count_youtube   3.774523
4           Like_Count_youtube   1.092866
5        Comment_Count_youtube        NaN
6   video_potentiality_youtube   3.654655
7          Category_ID_youtube   5.283367
8            Category_ID_games  45.178013
9       similarity_score_games        inf
10        Total Average Rating   1.000466
11                YouTube_Flag  31.148887
12                  Games_Flag        inf
13          Language_Game_Flag   1.623191
VIF가 낮은 독립변수들: ['View_Count_games', 'View_Count_youtube', 'Like_Count_youtube', 'video_potentiality_youtube', 'Category_ID_youtube', 'Total Average Rating', 'Language_Game_Flag']
Permanent Component - 1단계 OLS 결과
                                 OLS Regression Results                                 
Dep. Variable:     Permanent_Component   R-squared (uncenter

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 7
  warnings.warn('covariance of constraints does not have full '



Temporary Component - 1단계 OLS 결과
                                 OLS Regression Results                                 
Dep. Variable:     Temporary_Component   R-squared (uncentered):                   0.001
Model:                             OLS   Adj. R-squared (uncentered):              0.001
Method:                  Least Squares   F-statistic:                              21.51
Date:                 Sat, 19 Oct 2024   Prob (F-statistic):                    3.19e-29
Time:                         03:49:31   Log-Likelihood:                     -1.8703e+06
No. Observations:               292559   AIC:                                  3.741e+06
Df Residuals:                   292552   BIC:                                  3.741e+06
Df Model:                            7                                                  
Covariance Type:                   HC3                                                  
                                 coef    std err          z      P>|z|      

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 8, but rank is 7
  warnings.warn('covariance of constraints does not have full '
